In [1]:
import pandas as pd
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pickle
import mlflow





In [2]:
df = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
data2 = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")

# All Fucntions


In [6]:
def give_train_set(filename):
    df = pd.read_parquet(filename)
    df["duration"] = pd.to_datetime(df.tpep_dropoff_datetime) - pd.to_datetime(df.tpep_pickup_datetime)
    df["duration"] = df.duration.apply(lambda td: td.total_seconds()/60)
    pd.options.display.float_format = '{:.12f}'.format
    filtered = df[df.duration >= 1]
    filtered_furth =  filtered[filtered.duration <= 60]
    features = ["PULocationID", "DOLocationID"]
    filtered_furth[features] = filtered_furth[features].astype(str)
    
    dv = DictVectorizer()
    features = ["PULocationID", "DOLocationID"]
    train_dict = filtered_furth[features].to_dict(orient= 'records')
    x_train = dv.fit_transform(train_dict)
    y_train = filtered_furth["duration"].values
    return x_train, y_train,dv
    
    
    
def train_model(x_train, y_train, data):
    mlflow.set_tracking_uri("sqlite:///mlflow.db")
    mlflow.set_experiment("first experiment")
    with mlflow.start_run():
        mlflow.set_tag("developer", "akash")
        mlflow.log_param("train_path", data)
        fit_bool = True
        lr = LinearRegression(fit_intercept= fit_bool)
        lr.fit(x_train, y_train)
        pred = lr.predict(x_train)
        rmse = mean_squared_error(y_train, pred,squared = False) 
        mlflow.log_param("rmse", rmse)
        
    return lr
        
    
def predict_model(lr, x):
    pred = lr.predict(x)
    return pred
    
    

def score(actual, predicted):
    return mean_squared_error(actual, predicted,squared = False) 

# Creating Model

In [4]:
data = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet"
x_train, y_train,dv =  give_train_set(data)

/tmp/ipykernel_48245/1295326107.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_furth[features] = filtered_furth[features].astype(str)


In [7]:
model = train_model(x_train, y_train,data)

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [ ]:
predicted = predict_model(model, x_train)

In [ ]:
score =  score(y_train, predicted)

In [ ]:
score

## Save model

In [8]:
with open('lin_beg.bin', 'wb') as f_out:
    pickle.dump((dv,model), f_out)

# Experiment Tracking


In [9]:
import mlflow

In [10]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("first experiment")

<Experiment: artifact_location='/workspaces/practice_repo_zoom_camp/mlruns/1', creation_time=1719742145975, experiment_id='1', last_update_time=1719742145975, lifecycle_stage='active', name='first experiment', tags={}>

## Use hyperparamter and training it to find best parameter

In [11]:
import xgboost as xgb

In [12]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [13]:
train = xgb.DMatrix(x_train, label=y_train)
valid = xgb.DMatrix(x_train, label=y_train)

In [16]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1,
            evals=[(valid, 'validation')],
            early_stopping_rounds=1
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_train, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [17]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=1,
    trials=Trials()
)

  0%|                                                                                                         | 0/1 [00:00<?, ?trial/s, best loss=?]

/usr/local/python/3.10.13/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [10:59:03] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.05404                                                                                                                         
100%|████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.53s/trial, best loss: 9.054044984278436]


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



### once you get parameters you can use the paramters by using the paramters you got from above training